In [1]:
pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ultralytics import YOLO
from keras.models import load_model
import cv2
import numpy as np

In [3]:
od_model = YOLO('best.pt')
ic_model = load_model('Spine_Categorizer.h5')

In [5]:
#image = cv2.imread('7.png')

In [4]:
# Load the neuron image
neuron_image = cv2.imread('7.png')

# Upscale the neuron image using bicubic interpolation
#image = cv2.resize(neuron_image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)


In [5]:
detection_results = od_model(neuron_image)


0: 640x640 29 spines, 1253.3ms
Speed: 19.6ms preprocess, 1253.3ms inference, 3431.3ms postprocess per image at shape (1, 3, 640, 640)


In [6]:
type(detection_results)

list

In [10]:
#displaying the result

In [7]:
import matplotlib.pyplot as plt
import numpy as np

In [22]:
def display_image(image):
    plt.imshow(np.asarray(image))
    plt.axis('off')
    plt.show()



In [23]:
display_image(detection_results)

ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 32.

In [8]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Assuming ic_model is already loaded
# Assuming detection_results[0].boxes contains the bounding box information

# Define the target size
target_size = (250, 250)

# Iterate over detections
for det in detection_results[0].boxes:
    # Extract bounding box coordinates
    x1, y1, x2, y2 = [int(float(xy)) for xy in det.xyxy[0]]
    
    # Crop the detected spines from the original image
    crop_img = image[y1:y2, x1:x2]
    
    # Resize the cropped image while preserving the aspect ratio
    resized_crop_img = cv2.resize(crop_img, target_size, interpolation=cv2.INTER_AREA)
    
    # Display the resized cropped image
    plt.imshow(resized_crop_img, cmap='gray')  # Display in grayscale
    plt.axis('off')
    plt.show()

    print(resized_crop_img.shape)  # Print the shape of the resized image
    
    # Convert the resized cropped image to grayscale
    gray_resized_crop_img = cv2.cvtColor(resized_crop_img, cv2.COLOR_BGR2GRAY)
    
    # Expand dimensions to match the shape that the model expects
    input_crop_img = np.expand_dims(gray_resized_crop_img, axis=-1)

    # Run image classification on the cropped image
    ic_results = ic_model.predict(np.array([input_crop_img]))
    class_label = np.argmax(ic_results[0])  # Get the top-1 class label


NameError: name 'image' is not defined

In [ ]:
print(class_label)

In [ ]:
class_label_str = str(int(class_label))

In [ ]:
cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
cv2.putText(image, class_label_str, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

In [ ]:
import cv2
import numpy as np

# Assuming 'image' is your original image
# Assuming 'detection_results' contains the bounding box and class label information

# Make a copy of the original image to draw rectangles and labels on
image_with_rectangles_and_labels = image.copy()

# Iterate over detections
for det in detection_results[0].boxes:
    # Extract bounding box coordinates
    x1, y1, x2, y2 = [int(float(xy)) for xy in det.xyxy[0]]
    
    # Draw rectangle around the detected object
    cv2.rectangle(image_with_rectangles_and_labels, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Get the class label
    class_label_str = str(int(det.cls[0].item()))  # Convert class label to string
    
    # Draw label near the top-left corner of the rectangle
    cv2.putText(image_with_rectangles_and_labels, class_label_str, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image with rectangles and labels
cv2.imshow('Image with Rectangles and Labels', image_with_rectangles_and_labels)
cv2.waitKey(0)
cv2.destroyAllWindows()
